# Seq2Seq LSTM

![Seq2Seq](https://raw.githubusercontent.com/bentrevett/pytorch-seq2seq/eff9642693c3d83c497a791e80a34e740874f5cd/assets/seq2seq7.png)

The left part is an LST called `Ecoder`. It takes as input a german sentence.

It will produce the `z` red box, called `Context Vector`.

`z` will be the input of the right LSTM `Decoder`.

It takes a input and cell by cell it will produce and output translated sentence.
